# Working with Errors
Last chapter we started working on the implementation of the domain, but we neglected to mention errors.   
All systems have errors and we can make these explicit but hopefully handle them elegantly as well.  

## Using the `Result` type to make errors explicit
We don't want to have invisible control flow and `goto` statements in our code.   
In the previous chapter we threw exceptions which means that all of the function signatures were misleading.  
We can represent these errors in the type system to make it explicit that they can happen and force the users of our API to handle them (even if those users are us)

Here is an example from last chapter:

In [ ]:
type Undefined = exn
type UnvalidatedAddress = Undefined
type CheckedAddress = Undefined

module Bad = 
    type CheckAddressExists = UnvalidatedAddress -> CheckedAddress

module Clearer =
    type CheckAddressExists = UnvalidatedAddress -> Result<CheckedAddress, AddressValidationError>
        and 
        AddressValidationError =
        | InvalidFormat of string
        | AddressNotFound

Our function is now a total function (it returns an output for every input).
It tells us that:
- The input is an `UnvalidatedAddress`
- If the validation was successful, the output is a `CheckedAddress`
- If the validation was not successful, the reason is because the format was invalid or because the address was not found.

In this way the function's type signature can act as documentation.

## Working with Demain Errors
We can classify errors into 3 groups
1. Domain errors:  
    These are errors that are expected as part of the business process and must be included in the design. The business will already have procedures in place to deal with these errors and the code needs to reflect them.
1. Panics:  
    These are errors that leave the system in an unknown state such as unhandleable system errors or errors caused by programmer oversight.
1. Infrastructure errors:  
    These are errors that are expected, but are not part of any business process, such as network timeout or authentication failure.
  
We can determine if an error is a domain error by asking the domain expert (or not depending on how weird that would be)  

We handle these errors in different ways.

### Domain errors
Domain errors are part of the domain, and should be incorporated in the domain modelling, discussed with the domain experts and documented in the type system.

### Panics
Panics are best handled by abandoning the workflow and raising an exception which is caught at the highest appropriate level.

### Infrastructure errors
Infrastructure errors can be handled with either of the above approaches. It depends on the individual case.

## Modelling domain errors in types
In the same way that we haven't been using primitive types in our domain, we want to have domain specific error types.  
Generally we should model errors as a choice type, with a separate case for each kind of error.  
For example we might model errors in our order taking workflow like this:

In [ ]:
type ProductCode = Undefined
type RemoteServiceError = Undefined

type PlaceOrderError =
| ValidationError of string
| ProductOutOfStock of ProductCode
| RemoteServiceError of RemoteServiceError
// ... etc

The choice type acts as explicit documentation, but also ensures that expanding the error enum is safe, because the compiler ensures that all pattern matchers have a case for the new error type.  
Errors usually come up naturally as you develop a system, and these can be added as we go.

## Error handling makes your code ugly (why we love Monads (and Applicatives (and Functors)))
The 'nice' thing about exceptions is that they keep your happy path code looking clean.  
If we have to return errors in each step, we get some very nested and ugly looking code (especially if we don't have early return)

```clojure
(let [customer-info-result (create-name)]
    (if (not customer-info-result)
      {:error "failed to create name"}
      (let [shipping-address-result (create-valid-address)]
        (if (not shipping-address-result)
          {:error "failed to create a valid address"}
          (let [] ...)))))
```

And even in languages with early return, we end up with a majority of code devoted to error handling.  
How can we introduce error handling which preserving the elegance of our pipeline?

## Chaining `Result`-generating functions
To summarise a very useful analogy, we create a function connector, which, once we have returned one error, bypasses all other logic and just returns the error.  
This function has a lot of names and here are a couple:
- `bind`
- `flatMap`
- `SelectMany` (Just because we're in .NET)

We can fairly intuitively write this function if we write out the requirements.
1. If the input to the function is a success, pass the inner value to the next function in the chain
1. If the input is a failure, bypass the logic and return the error

In [ ]:
let bind f = function
| Ok x -> f x
| e -> e

Another useful function is one that turns a function that takes a basic value and returns a function which takes a result.  
This function is called `map` (or `Select`) and looks like this:

In [ ]:
let map f = function
| Ok x -> Ok (f x)
| e -> e

## Organizing the `Result` functions
We should put these functions in a module with the same name is the type

In [7]:
type MyResult<'Success, 'Failure> =
| Ok of 'Success
| Failure of 'Failure

module MyResult =
    let bind f = function
    | Ok x -> f x
    | Failure e -> Failure e

    let map f = function
    | Ok x -> Ok (f x)
    | Failure e -> Failure e

    // map can be written in terms of bind
    let map' f = bind (f >> Ok)

    // Bind is often written with this symbol 
    let (>>=) x f = bind f x

In [ ]:
type Apple = Apple
type Banana = Banana
type Cherry = Cherry
type Lemon = Lemon

type FunctionA = Apple -> MyResult<Banana, unit>
type FunctionB = Banana -> MyResult<Cherry, unit>
type FunctionC = Cherry -> MyResult<Lemon, unit>

let functionA : FunctionA = fun a -> Failure ()
let functionB : FunctionB = fun b -> Failure ()
let functionC : FunctionC = fun c -> Failure ()

let functionABC input =
    input
    |> functionA
    |> MyResult.bind functionB
    |> MyResult.bind functionC

Currently our system allows the success type to change the type of our `Success` case, but the `Failure` case is the same type all the way along.  
We're going to need a way to change the error types to make them compatible with one another.  
We can do this with a `map` function the same way we did with the `Success` case.  

In [ ]:
module MyResult =
    let mapError f = function
    | Ok x -> Ok x
    | Failure e -> Failure (f e)

type AppleError = AppleError
type BananaError = BananaError
type CherryError = CherryError

type FruitError =
    | AppleErrorCase of AppleError
    | BananaErrorCase of BananaError

let functionA = fun a -> Failure AppleError

let functionWithAFruitError input =
    input
    |> functionA
    |> MyResult.mapError AppleErrorCase

## Using `bind` and `map` in our pipeline
Let's see how we can use these functions in our pipeline

In [ ]:
module Domain =
    type UnvalidatedOrder = Undefined
    type ValidatedOrder = Undefined
    type ValidationError = Undefined
    type PricingError = Undefined
    type PricedOrder = Undefined
    type OrderAcknowledgementSent = Undefined
    type PlaceOrderEvent = Undefined

    type ValidateOrder = UnvalidatedOrder -> MyResult<ValidatedOrder, ValidationError>
    type PriceOrder = ValidatedOrder -> MyResult<PricedOrder, PricingError>
    type AcknowledgeOrder = PricedOrder -> OrderAcknowledgementSent option
    type CreateEvents = PricedOrder -> OrderAcknowledgementSent option -> PlaceOrderEvent list

    type PlaceOrderError = 
    | Validation of ValidationError
    | Pricing of PricingError

    let validateOrder: ValidateOrder = fun unvalidated -> failwith "not implemented"
    let priceOrder: PriceOrder = fun validated -> failwith "not implemented"
    let acknowledgeOrder: AcknowledgeOrder = fun priced -> failwith "not implemented"
    let createEvents: CreateEvents = fun priced -> failwith "not implemented"
    
    let validateOrderAdapted input =
        input 
        |> validateOrder
        |> MyResult.mapError PlaceOrderError.Validation


    let priceOrderAdapted input =
        input
        |> priceOrder
        |> MyResult.mapError PlaceOrderError.Pricing

    let placeOrder unvalidatedOrder =
        unvalidatedOrder
        |> validateOrderAdapted
        |> MyResult.bind priceOrderAdapted
        |> MyResult.map acknowledgeOrder
        // |> MyResult.map createEvents
        // This last line doesn't work


As you can see, the proposed last line doesn't work. This is because we're missing some adapter types.  
Our `MyResult.map acknowledgeOrder` function returns a `Result<OrderAcknowledgementSent option, PlaceOrderError>`  
but `MyResult.map createEvents` function takes input of type `Result<PricedOrder, PlaceOrderError>`

## Other adapter types
### Handling Exceptions
We aren't throwing exceptions in our domain code, but what about those thrown by libraries or other services?  
We can create another type of adapter which handles exceptions for us.

In [ ]:
type ServiceInfo = {
    Name: string
    Endpoint: Uri
}

type RemoteServiceError = {
    Service: ServiceInfo
    Exception: System.Exception
}

exception AuthorizationException

let serviceExceptionAdapter serviceInfo serviceFn x =
    try
        Ok (serviceFn x)
    with
    | :? TimeoutException as ex ->
        Failure { Service=serviceInfo; Exception=ex }
    | :? AuthorizationException as ex ->
        Failure {Service=serviceInfo; Exception=ex}

Note that we are only catching exceptions that are relevant to the domain.  

In [ ]:
let serviceInfo = {
    Name = "AddressCheckingService"
    Endpoint = Uri("http://google.com")
}

let checkAddressExists address = raise AuthorizationException

let checkAddressExistsR address =
    let adaptedService =
        serviceExceptionAdapter serviceInfo checkAddressExists
    adaptedService address

The type of our original function looked like this
```fsharp
checkAddressExists -> CheckedAddress
```
but our new type looks like this
```fsharp
checkAddressExistsR -> Result<CheckedAddress, RemoteServiceError>
```

As a side note we would have to update our PlaceOrderError enum to include the new error type
```fsharp
type PlaceOrderError =
| Validation of ValidationError
| Pricing of PricingError
| RemoteService of RemoteServiceError
```

### Handling dead-end functions
Another common type of function we call is a 'dead-end function'. A function which takes input and performs side effects but doesn't return any output.  
To make this work with the two track pipeline, we need another adapter.

In [ ]:
let tee f x = 
    f x
    x

let adaptDeadEnd f =
    Result.map (tee f)

Here we are running the function and returning the input.

## Making life easier with computation expressions
So far we've been able to join our expressions together fairly simply with `bind` and `map` but sometimes the flow is more complicated.  
We might need conditional branches, loops or deeply nested functions. F# provides some syntax sugar for this.  
They look like magic but under the hood it's surprisingly simple.

For the most basic computation expression, you need 2 functions (which look suspiciously like the definition of a monad...)
1. `bind` which we have seen already
1. `return` which just wraps a value. In this case, `return` is just the `Ok` constructor.

With these functions we can convert this code:
```fsharp
let placeOrder unvalidatedOrder =
    unvalidatedOrder
    |> validateOrderAdapted
    |> Result.bind priceOrderAdapted
    |> Result.map acknowledgeOrder
    |> Result.map createEvents
```
into
```fsharp
let placeOrder unvalidatedOrder = result {
    let! validatedOrder = validateOrder unvalidatedOrder |> Result.mapError PlaceOrderError.Validation
    let! pricedOrder = priceOrder validatedOrder |> Result.mapError PlaceOrderError.Pricing
    let acknowledgementOption = acknowledgeOrder pricedOrder
    let events = createEvents pricedOrder acknowledgementOption
    return events
}
```

We can imagine that this roughly desugars to
```fsharp
let (>>=) x f = bind f x

let placeOrder unvalidatedOrder =
    validateOrder unvalidatedOrder |> Result.mapError PlaceOrderError.Validation >>= (fun validatedOrder ->
    priceOrder validatedOrder |> Result.mapError PlaceOrderError.Pricing >>= (fun pricedOrder ->
    let acknowledgementOption = acknowledgeOrder pricedOrder
    let events = createEvents pricedOrder acknowledgementOption
    Ok events)) 
```

You can sort of see where the nesting has gone in our code. It's still there because it has to be, but we don't see it any more.

In [ ]:
type ResultBuilder() =
    member this.Return(x) = MyResult.Ok x
    member this.Bind(x, f) = MyResult.bind f x

let result = ResultBuilder()

open Domain

let validateOrder: ValidateOrder = fun unvalidated -> failwith "not implemented"
let priceOrder: PriceOrder = fun validated -> failwith "not implemented"

let validateOrder' unvalidatedOrder = result {
    let! validatedOrder = validateOrder unvalidatedOrder |> MyResult.mapError PlaceOrderError.Validation
    return validatedOrder
}
let priceOrder' validatedOrder = result {
    let! pricedOrder = priceOrder validatedOrder |> MyResult.mapError PlaceOrderError.Pricing
    return pricedOrder
}

Now lets apply this to some code that doesn't currently have any error handling.  
For example the validateOrder flow from the last chapter:

```fsharp
let validateOrder : ValidateOrder =
    fun checkProductCodeExists checkAddressExists unvalidatedOrder ->
        let orderId =
            unvalidatedOrder.OrderId
            |> OrderId.create
        let customerInfo =
            unvalidatedOrder.CustomerInfo
            |> toCustomerInfo
        let shippingAddress = 
            unvalidatedOrder.ShippingAddress
            |> toAddress checkAddressExists
        let billingAddress = ...
        let lines = ...

        let validatedOrder: ValidatedOrder = {
            OrderId = orderId
            CustomerInfo = customerInfo
            ShippingAddress = shippingAddress
            BillingAddress = billingAddress
            Lines = lines
        }

        validatedOrder
```

We just need to change all of the functions that can error to return `Result`s. For example now `OrderId.create` should return a `Result<OrderId, string>`.  
Now the implementation looks like this

```fsharp
let validateOrder: ValidateOrder =
    fun checkProductCodeExists checkAddressExists unvalidatedOrder -> result {
        let! orderId =
            unvalidatedOrder.OrderId
            |> OrderId.create
            |> MyResult.mapError ValidationError
        let! customerInfo =
            unvalidatedOrder.CustomerInfo
            |> toCustomerInfo
        let! shippingAddress = ...
        let! billingAddress = ...
        let! lines = ...

        let validatedOrder: ValidatedOrder = {
            OrderId = orderId
            CustomerInfo = customerInfo
            ShippingAddress = shippingAddress
            BillingAddress = billingAddress
            Lines = lines
        }

        return validatedOrder
    }
```

### Working with lists of results
Originally we could just map over the lines in our order, but now that the `toValidatedOrderLine` function returns a `Result` we end up with a `Result<ValidatedOrderLine, ...> list`  
We don't want a `Result<ValidatedOrderLine, ...> list` we would rather have a `Result<ValidatedOrderLine list, ...>`  
We can write a function that inverts this type. It is usually called `sequence`.

In [ ]:
module MyResult =
    let prepend firstR restR =
        match firstR, restR with
        | Ok first, Ok rest -> Ok (first::rest)
        | Failure err1, Ok _ -> Failure err1
        | Ok _, Failure err2 -> Failure err2
        | Failure err1, Failure _ -> Failure err1

    let sequence aListOfResults =
        let initialValue = Ok []
        // foldBack is equivalent to reduce
        List.foldBack prepend aListOfResults initialValue

Now we can write our validation code like this
```fsharp
let validateOrder: ValidateOrder =
    fun checkProductCodeExists checkAddressExists unvalidatedOrder -> result {
        ...
        let! lines = unvalidatedOrder.Lines
        |> List.map (toValidatedOrderLine checkProductCodeExists)
        |> MyResult.sequence
    }

    let validatedOrder: ValidatedOrder = {
            OrderId = orderId
            CustomerInfo = customerInfo
            ShippingAddress = shippingAddress
            BillingAddress = billingAddress
            Lines = lines
        }

    return validatedOrder
```

`List.map` followed by `Result.sequence` can be made more efficient by combining them into a single operation called `traverse`

## What is a Monad?
The book tries not to use the word Monad until this point, but a Monad is just a data structure with the `return` (or `pure`) function and a `bind` function.  
It is important that these functions follow some rules
1. Identity  
    `bind (return x) h ≡ h x`
1. Right identity  
    `bind return x ≡  x`
1. Associativity  
    `bind h (bind g x) ≡ bind (fun y -> bind h (g y)) x`

We have seen in this chapter how we can use these functions to hide side effects in normal looking code.

## Composing in parallel with Applicatives
Applicatives are similar to Monads, but rather than chaining functions in series, they allow chaining functions in parallel.  
All Monads are Applicative functors.  
Applicative validation allows us to run several validation functions, returning all of the errors that come up. This means if we get a validation error, we don't have to fix them one at a time and order doesn't matter.

In [3]:
module Applicatives =
    type UnvalidatedUser = {
        Name: string
        Email: string
        DateOfBirth: string
    }

    type ValidatedUser = {
        Name: string
        Email: string
        DateOfBirth: DateTime
    }

    type UserCreationFailure =
        | NameIsInvalidFailure
        | EmailIsInvalidFailure
        | DateOfBirthIsInvalidFailure
        | IDoNotLikeYouFailure

    let validateName : string -> Result<string, UserCreationFailure list> =
        fun input ->
            if not (String.IsNullOrWhiteSpace(input)) then Ok input else Error [ NameIsInvalidFailure ]

    let validateEmail : string -> Result<string, UserCreationFailure list> =
        fun input ->
            if input = "ben.cross@kroo.com" then Ok input else Error [ EmailIsInvalidFailure ]

    let validateDateOfBirth : string ->  Result<DateTime, UserCreationFailure list> =
        fun input ->
            if input = "1970-01-01" then Ok (DateTime(1970, 1 ,1)) else Error [ DateOfBirthIsInvalidFailure; IDoNotLikeYouFailure ]

    let create name email dateOfBirth =
        { ValidatedUser.Name = name; Email = email; DateOfBirth = dateOfBirth}

    module Result =
        let apply (fResult: Result<('a -> 'b), 'c list>) (xResult: Result<'a, 'c list>): Result<'b, 'c list> =
            match fResult, xResult with
            | Ok f, Ok x -> Ok (f x)
            | Error ex, Ok _ -> Error ex
            | Ok _, Error ex -> Error ex
            | Error ex1, Error ex2 -> Error (List.concat [ex1; ex2])

    let validate (input: UnvalidatedUser): Result<ValidatedUser, UserCreationFailure list> =
        let validatedName = validateName input.Name
        let validatedEmail = validateEmail input.Email
        let validatedDateOfBirth = validateDateOfBirth input.DateOfBirth
        create
        |> Result.map <| validatedName
        |> Result.apply <| validatedEmail
        |> Result.apply <| validatedDateOfBirth

Applicatives.validate { Name = "Ben"; Email = "ben.cross@kroo.com"; DateOfBirth = "1970-01-01"}
// Ok { Name = "Ben"; Email = "ben.cross@kroo.com"; DateOfBirth = DateTime(1970, 1, 1)}
Applicatives.validate { Name = ""; Email = "not.ben.cross@kroo.com!"; DateOfBirth = "Invalid date time"}
// Error [ NameIsInvalidFailure; EmailIsInvalidFailure; DateOfBirthIsInvalidFailure; IDoNotLikeYouFailure ]
        

Error: input.fsx (1,1)-(1,1) interactive error internal error: Could not load type 'UserCreationFailure' from assembly 'FSI-ASSEMBLY, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null'.

## Adding the Async Effect
In our pipeline we didn't just use `Result`, we also used `Async`.  
We can define a special `asyncResult` computation expression to deal with this case.  
There is a *lot* of code for this in the source for this book.  

The important part of how this changes the code we already have is if we have a function that returns a `Result` we have to lift it to an `AsyncResult` in order to use `let!`  
```fsharp
let validateOrder: ValidateOrder =
    fun checkProductCodeExists checkAddressExists unvalidatedOrder -> asyncResult {
        let! orderId =
            unvalidatedOrder.OrderId
            |> OrderId.create
            |> Result.mapError ValidationError
            |> AsyncResult.ofResult // lift Result to AsyncResult
        let! customerInfo =
            unvalidatedOrder.CustomerInfo
            |> toCustomerInfo
            |> AsyncResult.ofResult
        let! checkedShippingAddress =    
            unvalidatedOrder.ShippingAddress
            |> toCheckedAddress checkAddressExists // This returns an AsyncResult so we don't need to lift it.
        let! shippingAddress =
            unvalidatedOrder.shippingAddress
            |> toAddress
            |> AsyncResult.ofResult
        let! billingAddress = ...
        let! lines =
            unvalidatedOrder.Lines
            |> List.map (toValidatedOrderLine checkProductCodeExists)
            |> Result.sequence
            |> AsyncResult.ofResult

        let validatedOrder: ValidatedOrder = {
            OrderId = orderId
            CustomerInfo = customerInfo
            ShippingAddress = shippingAddress
            BillingAddress = billingAddress
            Lines = lines
        }
    }
```